In [1]:
from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
import numpy as np
from hyperas import optim
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD , Adam
import tensorflow as tf
from hyperas.distributions import choice, uniform

In [2]:
from pandas import read_csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# load dataset


In [3]:
dataframe = read_csv("C:/Users/ajjadhav/OneDrive - Nice Systems Ltd/Desktop/DataRobot/Sonar.csv", header=None)
dataset = dataframe.values

In [4]:
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [5]:
encoded_Y.shape

(208,)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, encoded_Y, test_size=0.33, random_state=42)

In [7]:
X_train.shape

(139, 60)

## HyperOpt

In [8]:
from hyperopt.pyll.base import scope 
from hyperopt import hp
#quniform returns float, some parameters require int; use this to force int
space = {'rate'      : hp.uniform('rate',0.01,0.5),
         'units'     : scope.int(hp.quniform('units',10,100,5)),
        'batch_size' : scope.int(hp.quniform('batch_size',100,250,25))}

In [9]:
space

{'rate': <hyperopt.pyll.base.Apply at 0x16f061828b0>,
 'units': <hyperopt.pyll.base.Apply at 0x16f06182a30>,
 'batch_size': <hyperopt.pyll.base.Apply at 0x16f06182b80>}

In [10]:
def f_nn(params):
    model = Sequential()
    model.add(Dense(units=params['units'], input_shape=(60,), activation='relu'))
    model.add(Dense(30,  activation='relu'))
    model.add(Dense(30,  activation='relu'))
    model.add(Dropout(rate=params['rate']))
    model.add(Dense(30,  activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    sgd = SGD(learning_rate=0.01, momentum=0.8)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    result = model.fit(X_train, y_train, 
                       verbose=0, 
                       validation_split=0.1,
                       batch_size=params['batch_size'],
                       epochs=200)
    validation_loss = np.amin(result.history['val_loss'])
    
    return {'loss': validation_loss, 
            'status': STATUS_OK, 
            'model': model, 
            'params': params}


In [11]:
from hyperopt import fmin

In [12]:
trials = Trials()
best = fmin(f_nn, 
            space, 
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)

100%|████████████████████████████████████████████████| 50/50 [03:51<00:00,  4.63s/trial, best loss: 0.4319424033164978]


In [13]:
best_model = trials.results[np.argmin([r['loss'] for r in trials.results])]['model']
best_params = trials.results[np.argmin([r['loss'] for r in trials.results])]['params']
worst_model = trials.results[np.argmax([r['loss'] for r in trials.results])]['model']
worst_params = trials.results[np.argmax([r['loss'] for r in trials.results])]['params']

In [14]:
best_params

{'batch_size': 100, 'rate': 0.37542997199294476, 'units': 80}

In [15]:
best_model.predict(X_test)[1:5]

3/3 [==============================] - 0s 1ms/step


array([[0.9378324 ],
       [0.33423606],
       [0.8566953 ],
       [0.11287879]], dtype=float32)